## Installation

In [11]:
# Install required packages
!pip install transformers datasets peft wandb evaluate rouge_score nltk

## Imports

In [12]:
import os
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    PeftModel,
    PeftConfig
)
import wandb
import evaluate
import nltk
from nltk.tokenize import sent_tokenize

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Dataset

In [14]:
# Load the dataset
dataset = load_dataset("Short-Answer-Feedback/saf_communication_networks_english")

# Examine a sample
print(dataset['train'][0])

# Get basic statistics
print(f"Train set size: {len(dataset['train'])}")
print(f"Validation set size: {len(dataset['validation'])}")
print(f"Test set size: {len(dataset['test_unseen_answers'])}")

{'id': '6a31b925382d4e31a417cc78399dbff2', 'question': 'What is "frame bursting"? Also, give 1 advantage and disadvantage compared to the carrier extension.', 'reference_answer': 'Frame bursting reduces the overhead for transmitting small frames by concatenating a sequence of multiple frames in one single transmission, without ever releasing control of the channel.\nAdvantage :it is more efficient than carrier extension as single frames not filled up with garbage.\nDisadvantage :need frames waiting for transmission or buffering and delay of frames', 'provided_answer': 'Frame bursting is a feature for the IEEE 802.3z standard.\nAdvantage: better efficiency\nDisadvantage: station has to wait for enough data to send so frames need to wait (n-to-n delay)', 'answer_feedback': 'The response correctly answers the advantage and disadvantage part of the question. However, the definition is missing in the answer. The correct definition is that frame bursting is used to concatenate a sequence of 

### Pre-processing

In [15]:
# Define the preprocessing function
def preprocess_function(examples):
    # Create inputs that include question, provided answer, and score
    inputs = [
        f"Question: {q}\nStudent Answer: {a}\nScore: {s}"
        for q, a, s in zip(examples["question"], examples["provided_answer"], examples["score"])
    ]

    # Get feedback as targets
    targets = examples["answer_feedback"]

    # Tokenize the inputs
    model_inputs = tokenizer(
        inputs,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    # Tokenize the targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            padding="max_length",
            truncation=True,
            max_length=256,
            return_tensors="pt"
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Apply preprocessing to the datasets
tokenized_train = dataset["train"].map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
tokenized_val = dataset["validation"].map(preprocess_function, batched=True, remove_columns=dataset["validation"].column_names)
tokenized_test = dataset["test_unseen_answers"].map(preprocess_function, batched=True, remove_columns=dataset["test_unseen_answers"].column_names)

# Set the format for PyTorch
tokenized_train.set_format("torch")
tokenized_val.set_format("torch")
tokenized_test.set_format("torch")

Map:   0%|          | 0/1700 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/427 [00:00<?, ? examples/s]

Map:   0%|          | 0/375 [00:00<?, ? examples/s]

## Evaluation metric

In [16]:
# Load the ROUGE and BLEU metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode predicted and reference texts
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]

    # Calculate ROUGE scores
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Calculate BLEU score
    result["bleu"] = bleu.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])["bleu"]

    # Add mean generated length
    prediction_lens = [len(pred.split()) for pred in decoded_preds]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Model

In [17]:
# Load the model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Verify the model is on the expected device
print(f"Model is on device: {next(model.parameters()).device}")

Model is on device: cuda:0


In [ ]:
# Initialize W&B
wandb.login(key="")
wandb.init()

# Define LoRA configuration for BART
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"]  # Target attention layers in BART
)

# Get the PEFT model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


eval/bleu,▁█
eval/gen_len,█▁
eval/loss,█▁
eval/rouge1,▁█
eval/rouge2,▁█
eval/rougeL,▁█
eval/rougeLsum,▁█
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,▁▁
train/epoch,▁▁▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇███


trainable params: 2,359,296 || all params: 408,649,728 || trainable%: 0.5773


## PEFT Fine-tuning using Lora

In [19]:
# Define data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8
)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results/bart-student-feedback",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    greater_is_better=True,
    push_to_hub=False,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=50,
    logging_first_step=True,
    report_to="wandb",
)

# Create Seq2SeqTrainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()

# Stop W&B
wandb.finish()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-19-4cd7848b13eb>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Gen Len
1,1.433800,1.051032,0.232200,0.082200,0.178900,0.201300,0.034800,51.522200
2,1.268300,0.987097,0.242800,0.103800,0.194500,0.214600,0.046800,53.278700
3,1.196300,0.962763,0.258900,0.120100,0.208500,0.229800,0.059600,52.695600
4,1.161800,0.946353,0.263300,0.128300,0.214300,0.233500,0.065900,52.288100
5,1.151500,0.937312,0.269100,0.136700,0.220100,0.239900,0.074700,52.838400
6,1.142700,0.929586,0.271800,0.141800,0.226100,0.245800,0.078700,51.913300
7,1.135800,0.926046,0.273100,0.144100,0.226100,0.246300,0.081200,51.915700
8,1.136700,0.922754,0.275800,0.149800,0.227700,0.250300,0.085900,52.861800
9,1.138000,0.921732,0.274900,0.148900,0.227200,0.248600,0.084200,53.281000
10,1.116700,0.920515,0.275900,0.148100,0.227600,0.248600,0.084600,52.955500


eval/bleu,▁▃▄▅▆▇▇███
eval/gen_len,▁█▆▄▆▃▃▆█▇
eval/loss,█▅▃▂▂▁▁▁▁▁
eval/rouge1,▁▃▅▆▇▇████
eval/rouge2,▁▃▅▆▇▇▇███
eval/rougeL,▁▃▅▆▇█████
eval/rougeLsum,▁▃▅▆▇▇▇███
eval/runtime,▁▅▅▄█▂▁▃▆▆
eval/samples_per_second,▇▄▄▅▁▇█▆▃▃
eval/steps_per_second,▇▄▄▅▁▇█▆▃▃
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇████


## Saving model adapters

In [26]:
# Save the model and adapters
model_save_path = "./results/bart-student-feedback/final"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('./results/bart-student-feedback/final/tokenizer_config.json',
 './results/bart-student-feedback/final/special_tokens_map.json',
 './results/bart-student-feedback/final/vocab.json',
 './results/bart-student-feedback/final/merges.txt',
 './results/bart-student-feedback/final/added_tokens.json',
 './results/bart-student-feedback/final/tokenizer.json')

## Evaluation

In [28]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel, PeftConfig

# Path to your saved model
model_save_path = "./results/bart-student-feedback/final"

# Load the PEFT configuration
config = PeftConfig.from_pretrained(model_save_path)

# Load the base model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the PEFT adapter weights
model = PeftModel.from_pretrained(model, model_save_path)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Custom test question and answer
custom_question = "What is the difference between TCP and UDP protocols?"
custom_answer = "TCP is connection-oriented and guarantees delivery of packets, while UDP is connectionless and doesn't guarantee delivery."
custom_score = 0.8  # Assuming a score between 0 and 1

# Generate feedback
def generate_feedback(question, answer, score):
    input_text = f"Question: {question}\nStudent Answer: {answer}\nScore: {score}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Move inputs to the same device as model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate feedback
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            early_stopping=True
        )

    # Decode the generated feedback
    generated_feedback = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_feedback

# Generate and print feedback
feedback = generate_feedback(custom_question, custom_answer, custom_score)

print("Custom Question:")
print(custom_question)
print("\nStudent Answer:")
print(custom_answer)
print("\nScore:")
print(custom_score)
print("\nGenerated Feedback:")
print(feedback)

Custom Question:
What is the difference between TCP and UDP protocols?

Student Answer:
TCP is connection-oriented and guarantees delivery of packets, while UDP is connectionless and doesn't guarantee delivery.

Score:
0.8

Generated Feedback:
The response is partially correct as it states the difference between TCP and UDP but does not explain why TCP is connection-oriented and UDP is connectionless. However, the explanation of the difference is not correct as TCP is not a connectionless protocol, it is just a TCP-based protocol.


In [29]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("Short-Answer-Feedback/saf_communication_networks_english")

# Path to your saved model
model_save_path = "./results/bart-student-feedback/final"

# Load the PEFT configuration
config = PeftConfig.from_pretrained(model_save_path)

# Load the base model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the PEFT adapter weights
model = PeftModel.from_pretrained(model, model_save_path)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Choose an example from the test set
test_example = dataset["test_unseen_answers"][0]  # Using the first example
question = test_example["question"]
answer = test_example["provided_answer"]
score = test_example["score"]
true_feedback = test_example["answer_feedback"]

# Generate feedback
def generate_feedback(question, answer, score):
    input_text = f"Question: {question}\nStudent Answer: {answer}\nScore: {score}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Move inputs to the same device as model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate feedback
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            early_stopping=True
        )

    # Decode the generated feedback
    generated_feedback = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_feedback

# Generate and print feedback
feedback = generate_feedback(question, answer, score)

print("Test Question:")
print(question)
print("\nStudent Answer:")
print(answer)
print("\nScore:")
print(score)
print("\nTrue Feedback:")
print(true_feedback)
print("\nGenerated Feedback:")
print(feedback)

Test Question:
State at least 4 of the differences shown in the lecture between the UDP and TCP headers.

Student Answer:
In TCP there is a Sequence Number field to identify packets individually for reliability. There is no Sequence Number in UDP. The UDP header does not have an options field, while the TCP header does. In TCP there is an Advertised Window field for the Sliding Window Protocol for Flow Control. There is no Flow Control and therefore no Advertised Window field in UDP. In TCP there there is only a Data Offset field that specifies the header length. In UDP the whole Packet Length is transmitted.

Score:
1.0

True Feedback:
The response correctly identifies four differences between TCP and UDP headers.

Generated Feedback:
The response correctly states four differences between TCP and UDP headers. The response also correctly identifies the Advertised Window field in TCP and the Data Offset field in UDP. Apart from that, the response is correct as it correctly identifies al

In [30]:
import torch
import random
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("Short-Answer-Feedback/saf_communication_networks_english")

# Path to your saved model
model_save_path = "./results/bart-student-feedback/final"

# Load the PEFT configuration
config = PeftConfig.from_pretrained(model_save_path)

# Load the base model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the PEFT adapter weights
model = PeftModel.from_pretrained(model, model_save_path)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Generate feedback
def generate_feedback(question, answer, score):
    input_text = f"Question: {question}\nStudent Answer: {answer}\nScore: {score}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Move inputs to the same device as model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate feedback
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            early_stopping=True
        )

    # Decode the generated feedback
    generated_feedback = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_feedback

# Get 5 random examples from the test set
test_set_size = len(dataset["test_unseen_answers"])
random_indices = random.sample(range(test_set_size), 5)

# Generate and print feedback for each random example
for i, idx in enumerate(random_indices):
    test_example = dataset["test_unseen_answers"][idx]
    question = test_example["question"]
    answer = test_example["provided_answer"]
    score = test_example["score"]
    true_feedback = test_example["answer_feedback"]

    # Generate feedback
    feedback = generate_feedback(question, answer, score)

    print(f"\n--- Example {i+1} (index {idx}) ---")
    print("Question:")
    print(question)
    print("\nStudent Answer:")
    print(answer)
    print("\nScore:")
    print(score)
    print("\nTrue Feedback:")
    print(true_feedback)
    print("\nGenerated Feedback:")
    print(feedback)
    print("-" * 80)


--- Example 1 (index 327) ---
Question:
Name the 3 service classes the Data Link Layer offers and explain the differences between the classes.

Student Answer:
- Unconfirmed Connectionless Service 
The Unconfirmed Connectionless Service sends data to the receiver, without announcing it (building up a connection) first in data frames without any flow control. Because of the missing connection and flow control, it is possible that complete data frames can get lost.  
- Confirmed Connectionless Service 
Wheras the confirmed connectionless service sends the data frames and waits for an acknowledgement of the corresponding recipient. If the recipient confirms the data frame, the next data frame is being sent. If the recipient doesn’t answer for a long time, the data frame is being resent. If for some reason, the ackknowledgement gets lost, the recipient will eventually get a data frame twice, and will not be able to detect the duplication. The correction has to be made on a higher level. I

In [33]:
import torch
import random
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("Short-Answer-Feedback/saf_communication_networks_english")

# Path to your saved model
model_save_path = "./results/bart-student-feedback/final"

# Load the PEFT configuration
config = PeftConfig.from_pretrained(model_save_path)

# Load the base model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the PEFT adapter weights
model = PeftModel.from_pretrained(model, model_save_path)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Generate feedback
def generate_feedback(question, answer, score):
    input_text = f"Question: {question}\nStudent Answer: {answer}\nScore: {score}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Move inputs to the same device as model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate feedback
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            early_stopping=True
        )

    # Decode the generated feedback
    generated_feedback = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_feedback

# Get 5 random examples from the test set
test_set_size = len(dataset["test_unseen_answers"])
random_indices = random.sample(range(test_set_size), 5)

# Generate and print feedback for each random example
for i, idx in enumerate(random_indices):
    test_example = dataset["test_unseen_answers"][idx]
    question = test_example["question"]
    answer = test_example["provided_answer"]
    score = test_example["score"]
    true_feedback = test_example["answer_feedback"]

    # Generate feedback
    feedback = generate_feedback(question, answer, score)

    print(f"\n--- Example {i+1} (index {idx}) ---")
    print("Question:")
    print(question)
    print("\nStudent Answer:")
    print(answer)
    print("\nScore:")
    print(score)
    print("\nTrue Feedback:")
    print(true_feedback)
    print("\nGenerated Feedback:")
    print(feedback)
    print("-" * 80)


--- Example 1 (index 114) ---
Question:
Consider a single server queueing system with a buffer of size 10. Let us assume that 9 packets arrive per second and 10 packets are served per second on an average. Assume you monitor the system for exactly one minute after the system reaches equilibrium. How many seconds would you expect the system to be in a state in which there are less than 10 packets waiting in the queue? You need to justify your answer by showing steps involved; calculations, however, need not be included. headers.

Student Answer:
Since the system reached an equilibrium the probabilities do not change anymore ( dp_n(t)/dt = 0 ). So it can be assumed that the queue is emptied by one package each second on average (10 served - 9 arrived).

Now assuming that the queue is full at obvservation start, it will be empty after 10 seconds, from which 9 seconds the queue has less packets then 10.

Score:
0.0

True Feedback:
The response is incorrect because the stated number of exp

In [32]:
import torch
import numpy as np
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from datasets import load_dataset
import evaluate
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
import nltk

# Download necessary NLTK data
nltk.download('punkt', quiet=True)

# Load metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

# Load the dataset
dataset = load_dataset("Short-Answer-Feedback/saf_communication_networks_english")
test_dataset = dataset["test_unseen_answers"]

# Path to your saved model
model_save_path = "./results/bart-student-feedback/final"

# Load the PEFT configuration
config = PeftConfig.from_pretrained(model_save_path)

# Load the base model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the PEFT adapter weights
model = PeftModel.from_pretrained(model, model_save_path)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)
model.eval()

# Process examples in batches
def process_batch(batch_examples, batch_size=8):
    questions = [example["question"] for example in batch_examples]
    answers = [example["provided_answer"] for example in batch_examples]
    scores = [example["score"] for example in batch_examples]
    true_feedbacks = [example["answer_feedback"] for example in batch_examples]

    # Generate inputs for all examples in the batch
    batch_inputs = []
    for q, a, s in zip(questions, answers, scores):
        input_text = f"Question: {q}\nStudent Answer: {a}\nScore: {s}"
        batch_inputs.append(input_text)

    # Tokenize all inputs
    tokenized_inputs = tokenizer(
        batch_inputs,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)

    # Generate all outputs
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=tokenized_inputs.input_ids,
            attention_mask=tokenized_inputs.attention_mask,
            max_length=256,
            num_beams=4,
            early_stopping=True
        )

    # Decode all outputs
    generated_feedbacks = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

    return generated_feedbacks, true_feedbacks

# Generate predictions for the entire test set in batches
BATCH_SIZE = 8
all_predictions = []
all_references = []

# Calculate number of batches
num_examples = len(test_dataset)
num_batches = (num_examples + BATCH_SIZE - 1) // BATCH_SIZE  # Ceiling division

print(f"Generating predictions for {num_examples} examples in {num_batches} batches...")

for i in tqdm(range(0, num_examples, BATCH_SIZE)):
    # Get batch of examples
    batch_end = min(i + BATCH_SIZE, num_examples)
    batch_examples = [test_dataset[j] for j in range(i, batch_end)]

    # Process batch
    batch_predictions, batch_references = process_batch(batch_examples, BATCH_SIZE)

    # Add to overall lists
    all_predictions.extend(batch_predictions)
    all_references.extend(batch_references)

# Format for ROUGE - expects a newline after each sentence
formatted_predictions = ["\n".join(sent_tokenize(pred.strip())) for pred in all_predictions]
formatted_references = ["\n".join(sent_tokenize(ref.strip())) for ref in all_references]

# Calculate ROUGE scores
rouge_results = rouge.compute(
    predictions=formatted_predictions,
    references=formatted_references,
    use_stemmer=True
)

# Calculate BLEU score
bleu_results = bleu.compute(
    predictions=all_predictions,
    references=[[ref] for ref in all_references]
)

# Print results
print("\nEvaluation Results on Test Set:")
print(f"ROUGE-1: {rouge_results['rouge1'] * 100:.2f}%")
print(f"ROUGE-2: {rouge_results['rouge2'] * 100:.2f}%")
print(f"ROUGE-L: {rouge_results['rougeL'] * 100:.2f}%")
print(f"BLEU: {bleu_results['bleu'] * 100:.2f}%")

# Add additional statistics
prediction_lens = [len(pred.split()) for pred in all_predictions]
reference_lens = [len(ref.split()) for ref in all_references]

print("\nAdditional Statistics:")
print(f"Average prediction length: {np.mean(prediction_lens):.2f} words")
print(f"Average reference length: {np.mean(reference_lens):.2f} words")
print(f"Number of test examples: {len(all_predictions)}")

Using device: cuda
Generating predictions for 375 examples in 47 batches...


100%|██████████| 47/47 [02:02<00:00,  2.61s/it]



Evaluation Results on Test Set:
ROUGE-1: 28.14%
ROUGE-2: 15.81%
ROUGE-L: 23.54%
BLEU: 9.02%

Additional Statistics:
Average prediction length: 53.88 words
Average reference length: 17.16 words
Number of test examples: 375
